In [1]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
import spacy

In [2]:
STOCK_NEWS_DATASET_FILE = "../data/aapl_us_equities_news.csv"

In [3]:
df_content = pd.read_csv(STOCK_NEWS_DATASET_FILE)
nlp = spacy.load('en_core_web_sm') # Load spacy model

In [4]:
def clean_txt(txt):

    """
    Input: Text

    Remove every non-alphanumeric characters
    Lowercase every word except 'AAPL'

    Output: Text
    """

    # Onderstaande twee regels van https://towardsdatascience.com/nlp-text-preprocessing-steps-tools-and-examples-94c91ce5d30
    txt = re.sub(r'\W+', ' ', txt) # Vervant alle non-alphanumeric characters 1 of meer met een spatie. 
    txt = txt.replace("[^a-zA-Z]", " ") # Vervangt alle niet letters in spaties
    
    words = [word.lower() if word != "AAPL" else word for word in txt.split(' ')] # Capslocks verwijderen voor alle woorden behalve AAPL
    txt_temp = " ".join(words) # Join losse woorden weer samen tot 1 text    


    
    return txt_temp

In [5]:
def remove_stopwords(text):

    """
    Input: Text
    Remove stopwords and lemmatize text.
    Output: Text
    """

    return " ".join(token.lemma_ for token in nlp(text) if not token.is_stop)

In [6]:
# Execute functions in sequential order

clean_text = []

for i, text in df_content.content.iteritems():
    try: 
        text_temp = clean_txt(text)
        text_temp2 = remove_stopwords(text_temp)

        df_content.content[i] = text_temp2
    except:
        print(text)


C:\Users\groot\AppData\Local\Temp\ipykernel_9632\92903875.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_content.content[i] = text_temp2


In [7]:
df_content.to_csv('../data/Preprocessed_Newsarticles_1okt2022_V2.csv')

### NOTES ###

In [67]:
# Lemma even testen
token = 'riding'
[token.lemma_ for token in nlp(token)]

['ride']

In [70]:
# New stop words list 
customize_stop_words = [
    ':)'
]

# Mark them as stop words
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True

### Alert zijn op volgende punten:
* US wordt getokenized als u en s
